In [38]:
import os
import fitz
import re
from glob import glob
import pandas as pd
import tarfile

In [39]:
### From parse_in_rolls

COLUMNS = ["number","id", "elector_name", "father_or_husband_name", "relationship", "house_no", "age", "sex", "ac_name", "parl_constituency", "part_no", "year", "state", "filename", "main_town", "police_station", "mandal", "revenue_division", "district", "pin_code", "polling_station_name", "polling_station_address", "net_electors_male", "net_electors_female", "net_electors_third_gender", "net_electors_total","original_or_amendment"]

In [40]:
INPUT_FILE = '/opt/data/in-electoral-rolls/haryana.tar.gz'

In [41]:
rows = []
with tarfile.open(INPUT_FILE, "r:gz") as tar:
    members = tar.getmembers()[1:]
    print('Total PDF files: %d' % len(members))
    for i, t in enumerate(members):
        with tar.extractfile(t) as f:
            pdf_data = f.read()
        doc = fitz.open('pdf', pdf_data)
        for p in doc:
            text = p.getText()
            for m in re.finditer(r'(?<=\n)(.*?\n\d+\n.*?\nनाम :(.*?\nहै।))', '\n' + text, flags=re.DOTALL):
                box = m.group(1)
                elector_name = re.findall(r'(.*?)\d+', box, flags=re.S)[0].strip().replace('\n', ' ')
                number = re.findall(r'.*?(\d+)', box, flags=re.S)[0].strip()
                _id = re.findall(r'.*?\d+\n(.*?)\n', box, flags=re.S)[0].strip()
                a = re.findall(r'नाम :\n(.*?)\nमकान  नं. :', box, flags=re.S)[0].strip().split(':')
                relationship = a[0].strip()
                father_or_husband_name = a[1].strip()
                house_no = re.findall(r'मकान  नं. :(.*?)आयु', box, flags=re.S)[0].strip()
                age = re.findall(r'आयु :(.*?)िलग', box, flags=re.S)[0].strip()
                sex = re.findall(r'लग : (.*?)फोटो', box, flags=re.S)[0].strip()
                data = {'filename': os.path.basename(fn),
                        'number': number,
                        'id': _id,
                        'elector_name': elector_name,
                        'father_or_husband_name': father_or_husband_name,
                        'relationship': relationship,
                        'house_no': house_no,
                        'age': age,
                        'sex': sex,
                       }
                rows.append(data)
        print(i + 1, fn, len(rows))
        #break

Total PDF files: 17018
1 Haryana/haryana_pdfs/CMB0070145.PDF 1014


In [42]:
df = pd.DataFrame(rows)
df

,filename,number,id,elector_name,father_or_husband_name,relationship,house_no,age,sex
0,CMB0070145.PDF,1,HR/01/03/0537002,मुखतारी,झन्डू,पित,1,77,मिहला
1,CMB0070145.PDF,2,HR/01/03/0537005,मान िसंह,झन्डु,िपता,1,48,पुरुष
2,CMB0070145.PDF,3,MVB1341999,जीत रानी,मान िसह,पित,1,43,मिहला
3,CMB0070145.PDF,4,UMK0693887,सािहल,मान िसह,िपता,1,21,पुरुष
4,CMB0070145.PDF,5,UMK0693895,जीतेश,मान िसह,िपता,1,21,पुरुष
...,...,...,...,...,...,...,...,...,...
1009,CMB0070145.PDF,1010,UMK0210617,अमरजीत कौर,रघुबीर िसह,पित,34/2,38,मिहला
1010,CMB0070145.PDF,1011,UMK0769703,शीतल रानी,कुनाल सहगल,पित,54,26,मिहला
1011,CMB0070145.PDF,1012,UMK0753459,सोहन लाल,जोिगन्दर् िसह,िपता,913,29,पुरुष
1012,CMB0070145.PDF,1013,UMK0347989,सुिमत कु मार,राम कुमार,िपता,2011,25,पुरुष


In [5]:
df.relationship.unique()

array(['िपता', 'पित', 'माता', '......'], dtype=object)

In [6]:
df.sex.unique()

array(['पुरुष', 'मिहला'], dtype=object)

In [7]:
df.age.unique()

array(['52', '34', '31', '32', '27', '26', '25', '24', '23', '28', '57',
       '35', '47', '44', '51', '40', '60', '58', '64', '62', '54', '38',
       '45', '41', '59', '37', '56', '50', '36', '74', '68', '43', '76',
       '72', '66', '46', '61', '48', '42', '71', '39', '55', '75', '67',
       '49', '30', '53', '33', '29', '63', '69', '65', '77', '20', '81',
       '80', '70', '22', '89', '84', '73', '85', '21', '79', '83', '87',
       '19', '82', '78', '99', '95', '90', '92', '91', '94', '93', '96',
       '88', '86', '97', '100', '102', '108', '112', '98', '101', '103',
       '111', '113'], dtype=object)

In [8]:
df.to_csv('haryana.csv.tar.gz', index_label='index')

In [9]:
text

'नामावली पर्रुप\nनामावली पहचान\nिनवार्चक� की संख्या\nपुरुष\nमिहला\nअन्य\nकुल\nमूल पर्ित \nमूल नामावली\nनये पिरसीिमत िनवार्चन क्षेतर्� के िवस्तारानुसार \nवषर्  2017 तक के सभी अनुपूरक� सिहत एकीकृत \nवषर् 2018 की फोटो युक्त िनवार्चक नामावली\n455\n437\n0\n892\n455\n437\n0\n892\nिनवार्चक पंजीयन अिधकारी �ारा जारी \nकुल  33  पृ�� का पृ�  33\n'

In [10]:
for b in p.getText("rawdict")["blocks"]:
    for c in b:
        if c == 'lines':
            for l in b[c]:
                for s in l["spans"]:
                    for char in s["chars"]: print(char["c"], ord(char["c"]))

न 2344
ा 2366
म 2350
ा 2366
व 2357
ल 2354
ी 2368
  32
प 2346
र 2352
् 2381
र 2352
ु 2369
प 2346
न 2344
ा 2366
म 2350
ा 2366
व 2357
ल 2354
ी 2368
  32
प 2346
ह 2361
च 2330
ा 2366
न 2344
ि 2367
न 2344
व 2357
ा 2366
र 2352
् 2381
च 2330
क 2325
� 65533
  32
क 2325
ी 2368
  32
स 2360
ं 2306
ख 2326
् 2381
य 2351
ा 2366
प 2346
ु 2369
र 2352
ु 2369
ष 2359
म 2350
ि 2367
ह 2361
ल 2354
ा 2366
अ 2309
न 2344
् 2381
य 2351
क 2325
ु 2369
ल 2354
म 2350
ू 2370
ल 2354
  32
प 2346
र 2352
् 2381
ि 2367
त 2340
  32
म 2350
ू 2370
ल 2354
  32
न 2344
ा 2366
म 2350
ा 2366
व 2357
ल 2354
ी 2368
न 2344
य 2351
े 2375
  32
प 2346
ि 2367
र 2352
स 2360
ी 2368
ि 2367
म 2350
त 2340
  32
ि 2367
न 2344
व 2357
ा 2366
र 2352
् 2381
च 2330
न 2344
  32
क 2325
् 2381
ष 2359
े 2375
त 2340
र 2352
् 2381
� 65533
  32
क 2325
े 2375
  32
ि 2367
व 2357
स 2360
् 2381
त 2340
ा 2366
र 2352
ा 2366
न 2344
ु 2369
स 2360
ा 2366
र 2352
  32
व 2357
ष 2359
र 2352
् 2381
  32
  32
2 50
0 48
1 49
7 55
  32
त 2340
क 2325
  32
क 2325
े 2375
  32

In [11]:
fitz.version

('1.18.4', '1.18.0', '20201119085623')

In [12]:
for b in p.getText("rawdict")["blocks"]:
    print(b)

{'number': 0, 'type': 1, 'bbox': (45.2400016784668, 20.52001953125, 71.63999938964844, 38.16001892089844), 'width': 440, 'height': 294, 'ext': 'png', 'colorspace': 0, 'xres': 96, 'yres': 96, 'bpc': 1, 'image': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\xb8\x00\x00\x01&\x08\x00\x00\x00\x005\xcd\xda\\\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00\x08\xb1IDATx\x9c\xed\x9c\xedv\xdb \x10D\x93\xf7\x7f\xe8\xf6\x9c\xb6I\x1d\xdb\xc0~\xcc\xaev\xec\xb9\xbf\xc3\x0ep%\x19\x81\xda\xcf\x0fA\xc9\xe7\xd5\x1d\x101$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\x918R$\x8e\x14\x89#E\xe2H\

In [13]:
for c in b:
    print(b[c])

72
0
(223.97999572753906, 796.2898559570312, 563.9927368164062, 811.045166015625)
[{'spans': [{'size': 10.5, 'flags': 0, 'font': 'ArialUnicodeMS', 'color': 0, 'chars': [{'origin': (223.97999572753906, 808.19970703125), 'bbox': (223.97999572753906, 796.976806640625, 227.33302307128906, 811.045166015625), 'c': 'ि'}, {'origin': (227.337890625, 808.19970703125), 'bbox': (227.337890625, 796.976806640625, 233.485107421875, 811.045166015625), 'c': 'न'}, {'origin': (233.48878479003906, 808.19970703125), 'bbox': (233.48878479003906, 796.976806640625, 239.70265197753906, 811.045166015625), 'c': 'व'}, {'origin': (239.7131805419922, 808.19970703125), 'bbox': (239.7131805419922, 796.976806640625, 243.0662078857422, 811.045166015625), 'c': 'ा'}, {'origin': (243.05953979492188, 808.19970703125), 'bbox': (232.55953979492188, 796.976806640625, 243.05953979492188, 811.045166015625), 'c': 'र'}, {'origin': (243.05953979492188, 808.19970703125), 'bbox': (232.55953979492188, 796.976806640625, 243.0595397949

In [14]:
fitz.version

('1.18.4', '1.18.0', '20201119085623')

In [15]:
pd.DataFrame(doc[32].getFontList()).sort_values(3)

,0,1,2,3,4,5
1,369,n/a,TrueType,ArialMT,TT5,WinAnsiEncoding
0,365,ttf,Type0,IAGCAL+ArialUnicodeMS,TT1,Identity-H


In [16]:
page = doc[32]

In [17]:
for f in page.getFontList(): print(f)

(365, 'ttf', 'Type0', 'IAGCAL+ArialUnicodeMS', 'TT1', 'Identity-H')
(369, 'n/a', 'TrueType', 'ArialMT', 'TT5', 'WinAnsiEncoding')


In [18]:
print(doc._getXrefString(369))

<<
  /BaseFont /ArialMT
  /Encoding /WinAnsiEncoding
  /FirstChar 48
  /FontDescriptor 368 0 R
  /LastChar 57
  /Subtype /TrueType
  /Type /Font
  /Widths [ 556 0 556 556 556 556 0 556 556 556 ]
>>


In [19]:
doc._getXrefString(369) 

'<<\n  /BaseFont /ArialMT\n  /Encoding /WinAnsiEncoding\n  /FirstChar 48\n  /FontDescriptor 368 0 R\n  /LastChar 57\n  /Subtype /TrueType\n  /Type /Font\n  /Widths [ 556 0 556 556 556 556 0 556 556 556 ]\n>>'

In [21]:
import PyPDF2

In [22]:
reader = PyPDF2.PdfFileReader('../../parse_in_rolls/data/haryana/CMB0890088.PDF')

In [23]:
reader.numPages

38

In [24]:
reader.getPage(36).extractText()

'\n\n\n\n\n\n\n˝ò\n\n\n\n\n\n\n\n\n˝ò\n\n\n\n\n\n\n\n\n\n\n\n˝ò\n\n˙c\n\n\n\n\n\n\n\n\n\n˝ò\n\n\n\n\n\n\n\n\n\n\n\n˝ò\n\n\n\n\n\n\n\n\n\n˝ò\n\n˙c\n\n\n\n˝v˝ò\n\n\n\n˝ò\n\n\n\n\n\n\n\n\n\n\n˝ò\n\n\n\n\n\n˙E\n\n\n\n'

In [26]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):

    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams(char_margin=0.1))
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for i, page in enumerate(PDFPage.create_pages(doc)):
            if i > 35:
                interpreter.process_page(page)

    return(output_string.getvalue())

                
def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename


def split_to_title_and_pagenum(table_of_contents_entry):
    title_and_pagenum = table_of_contents_entry.strip()
    
    title = None
    pagenum = None
    
    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -2
            while title_and_pagenum[i].isdigit():
                i -= 1

            title = title_and_pagenum[:i].strip()
            pagenum = int(title_and_pagenum[i:].strip())
        
    return title, pagenum
    

In [27]:
text = convert_pdf_to_string('../../parse_in_rolls/data/haryana/CMB0890088.PDF')
text

'1021\n\nRRC0026880\nिनिध\nगु(cid:220)ता\nअिनल गु(cid:7385)ा\n443\n\nनाम :\nिपता :\nमकान  नं. : \n32\nआयु :\n\nिलग :\n\nमिहला\n\n1022\n\nRRC0452680\nराम\nसुमेर\nपांडे\nगया पर्साद\n974\n\nनाम :\nिपता :\nमकान  नं. : \n48\nआयु :\n\nिलग :\n\nपुरुष\n\nफोटो    \nउपलब्ध    \nहै।\n\n1023\n\nRRC0452664\nगीता\nदेवी\nराम सुमेर पांडे\n\nफोटो    \nउपलब्ध    \nहै।\n\nनाम :\nपित :\nमकान  नं. : \n46\nआयु :\n\n974\n\nिलग :\n\nमिहला\n\n1024\n\nRRC0452672\n\nिकशन\nमार\nकु\nराम सुमेर पांडे\n\nपांडे\n\n974\n\nनाम :\nिपता :\nमकान  नं. : \n27\nआयु :\n\nिलग :\n\nपुरुष\n\n1025\n\nRRC0223073\nगौरव\nवधवा\nसुभाष चन्द\n\nफोटो    \nउपलब्ध    \nहै।\n\nनाम :\nिपता :\nमकान  नं. : \n30\nआयु :\n\n1311\n\nिलग :\n\nपुरुष\n\n1026\n\nRRC1115294\n\nफोटो    \nउपलब्ध    \nहै।\n\nपूजा\nनैन िसह\n\nनाम :\nपित :\nमकान  नं. : \n30\nआयु :\n\n1519\n\nिलग :\n\nमिहला\n\nफोटो    \nउपलब्ध    \nहै।\n\nफोटो    \nउपलब्ध    \nहै।\n\n1027\n\nRRC0723064\nगु(cid:220)ता\nपा(cid:509)ल\nमह(cid:7286)दर िसह\n\nनाम :\nिपता :\nमकान  नं. : \n31\nआयु :\

In [28]:
for l in text.split('\n'):
    print(l)

1021

RRC0026880
िनिध
गु(cid:220)ता
अिनल गु(cid:7385)ा
443

नाम :
िपता :
मकान  नं. : 
32
आयु :

िलग :

मिहला

1022

RRC0452680
राम
सुमेर
पांडे
गया पर्साद
974

नाम :
िपता :
मकान  नं. : 
48
आयु :

िलग :

पुरुष

फोटो    
उपलब्ध    
है।

1023

RRC0452664
गीता
देवी
राम सुमेर पांडे

फोटो    
उपलब्ध    
है।

नाम :
पित :
मकान  नं. : 
46
आयु :

974

िलग :

मिहला

1024

RRC0452672

िकशन
मार
कु
राम सुमेर पांडे

पांडे

974

नाम :
िपता :
मकान  नं. : 
27
आयु :

िलग :

पुरुष

1025

RRC0223073
गौरव
वधवा
सुभाष चन्द

फोटो    
उपलब्ध    
है।

नाम :
िपता :
मकान  नं. : 
30
आयु :

1311

िलग :

पुरुष

1026

RRC1115294

फोटो    
उपलब्ध    
है।

पूजा
नैन िसह

नाम :
पित :
मकान  नं. : 
30
आयु :

1519

िलग :

मिहला

फोटो    
उपलब्ध    
है।

फोटो    
उपलब्ध    
है।

1027

RRC0723064
गु(cid:220)ता
पा(cid:509)ल
मह(cid:7286)दर िसह

नाम :
िपता :
मकान  नं. : 
31
आयु :

5418

िलग :

मिहला

1028

RRC0972976
नंदन

फोटो    
उपलब्ध    
है।

(cid:230)याम
जागे(cid:7393)र गु(cid:7385)ा
8154

नाम :
िपता :
मकान  नं. : 
54
आयु :


In [29]:
df2 = pd.read_csv('../../parse_in_rolls/csvs/Haryana_final.csv')
df2

,Unnamed: 0,number,id,elector_name,father_or_husband_name,relationship,house_no,age,sex,ac_name,...,revenue_division,district,pin_code,polling_station_name,polling_station_address,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total,original_or_amendment
0,0,1.0,HR/06/52/0483005,अशोक HAR Ca,हरी चन्द जायजा,father,1.0,58.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
1,1,2.0,HR/06/52/0483006,सविता देवी पट,अशोक कुमार उपलब्ध,husband,1.0,54.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
2,2,3.0,RRC0574285,दीपा me,अशोक कुमार —,father,1.0,26.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
3,3,4.0,JWP3659604,सत्य पाल ट्टेजा पे,राम टुटेजा a,father,2.0,67.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
4,4,9.0,RRC0722108,दीपक शमी कप,अशोक शर्मा उपलब्ध,father,2.0,36.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23963,821,904.0,IGNO150219,सोनिया कप,औम प्रकाश er,father,1658.0,33.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23964,822,905.0,IGN0O150201,संदीप कप,औम प्रकाश et,father,1658.0,31.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23965,823,NaN,906 IGN0471680,रेखा om,भूपेन्द्र सिंह a,father,1979.0,25.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23966,824,907.0,IGN0471698,सचिन me,राधे श्याम उपलब्ध,father,2126.0,22.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original


In [30]:
df2.id = df2.id.str.strip()
df2

,Unnamed: 0,number,id,elector_name,father_or_husband_name,relationship,house_no,age,sex,ac_name,...,revenue_division,district,pin_code,polling_station_name,polling_station_address,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total,original_or_amendment
0,0,1.0,HR/06/52/0483005,अशोक HAR Ca,हरी चन्द जायजा,father,1.0,58.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
1,1,2.0,HR/06/52/0483006,सविता देवी पट,अशोक कुमार उपलब्ध,husband,1.0,54.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
2,2,3.0,RRC0574285,दीपा me,अशोक कुमार —,father,1.0,26.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
3,3,4.0,JWP3659604,सत्य पाल ट्टेजा पे,राम टुटेजा a,father,2.0,67.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
4,4,9.0,RRC0722108,दीपक शमी कप,अशोक शर्मा उपलब्ध,father,2.0,36.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23963,821,904.0,IGNO150219,सोनिया कप,औम प्रकाश er,father,1658.0,33.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23964,822,905.0,IGN0O150201,संदीप कप,औम प्रकाश et,father,1658.0,31.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23965,823,NaN,906 IGN0471680,रेखा om,भूपेन्द्र सिंह a,father,1979.0,25.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23966,824,907.0,IGN0471698,सचिन me,राधे श्याम उपलब्ध,father,2126.0,22.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original


In [31]:
df2.elector_name = df2.elector_name.str.strip()
df2

,Unnamed: 0,number,id,elector_name,father_or_husband_name,relationship,house_no,age,sex,ac_name,...,revenue_division,district,pin_code,polling_station_name,polling_station_address,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total,original_or_amendment
0,0,1.0,HR/06/52/0483005,अशोक HAR Ca,हरी चन्द जायजा,father,1.0,58.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
1,1,2.0,HR/06/52/0483006,सविता देवी पट,अशोक कुमार उपलब्ध,husband,1.0,54.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
2,2,3.0,RRC0574285,दीपा me,अशोक कुमार —,father,1.0,26.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
3,3,4.0,JWP3659604,सत्य पाल ट्टेजा पे,राम टुटेजा a,father,2.0,67.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
4,4,9.0,RRC0722108,दीपक शमी कप,अशोक शर्मा उपलब्ध,father,2.0,36.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23963,821,904.0,IGNO150219,सोनिया कप,औम प्रकाश er,father,1658.0,33.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23964,822,905.0,IGN0O150201,संदीप कप,औम प्रकाश et,father,1658.0,31.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23965,823,NaN,906 IGN0471680,रेखा om,भूपेन्द्र सिंह a,father,1979.0,25.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23966,824,907.0,IGN0471698,सचिन me,राधे श्याम उपलब्ध,father,2126.0,22.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original


In [32]:
df2[df2.id.isin(df.id.unique())]

,Unnamed: 0,number,id,elector_name,father_or_husband_name,relationship,house_no,age,sex,ac_name,...,revenue_division,district,pin_code,polling_station_name,polling_station_address,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total,original_or_amendment
0,0,1.0,HR/06/52/0483005,अशोक HAR Ca,हरी चन्द जायजा,father,1.0,58.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
1,1,2.0,HR/06/52/0483006,सविता देवी पट,अशोक कुमार उपलब्ध,husband,1.0,54.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
2,2,3.0,RRC0574285,दीपा me,अशोक कुमार —,father,1.0,26.0,महिला,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
3,3,4.0,JWP3659604,सत्य पाल ट्टेजा पे,राम टुटेजा a,father,2.0,67.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
4,4,9.0,RRC0722108,दीपक शमी कप,अशोक शर्मा उपलब्ध,father,2.0,36.0,पुरूष,089 - फरीदाबाद,...,शून्य,फरीदाबाद,121002,088-फरीदाबाद शहर,"चांदीवाली धर्मशाला, फरीदाबाद ओल्ड (दांया कक्ष)",509,519,0,1028,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23953,811,894.0,IGN0302794,शिव शंकर सिंह कप,रणेन्द्र प्रताप सिंह a,father,1234.0,48.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23955,813,896.0,IGN0471664,शिम्पी me,जाहिद खान me,father,1244.0,24.0,NaN,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23962,820,903.0,IGN0150193,सन्‍्तोष कप,औम प्रकाश ला,husband,1658.0,61.0,महिला,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original
23966,824,907.0,IGN0471698,सचिन me,राधे श्याम उपलब्ध,father,2126.0,22.0,पुरूष,001 - कालका,...,शून्य,पंचकुला,133301,078-पिंजौर,कार्यालय वन मण्डल अधिकारी मोरनी पिंजौर फोरैस्...,479,429,0,908,original


In [33]:
df3 = df.merge(df2, on='id')
df3

,filename_x,number_x,id,elector_name_x,father_or_husband_name_x,relationship_x,house_no_x,age_x,sex_x,Unnamed: 0,...,revenue_division,district,pin_code,polling_station_name,polling_station_address,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total,original_or_amendment
0,CMB0880189.PDF,1,BFS2536829,सुरेश चंद,रतन िसह,िपता,.,52,पुरुष,0,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
1,CMB0880189.PDF,2,BFS2536803,मोनू,रतन िसह,िपता,.,34,पुरुष,1,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
2,CMB0880189.PDF,3,BFS2536795,शिश,रतन िसह,िपता,.,31,मिहला,2,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
3,CMB0880189.PDF,4,BFS2536811,अिनल,रतन िसह,िपता,.,31,पुरुष,3,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
4,CMB0880189.PDF,5,NAH1151406,रेनू भार�वाज,दीपक भार�ाज,पित,0,32,मिहला,4,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19195,CMB0690082.PDF,887,BTX1661552,राजेश,नरेन्दर्,पित,225क,41,मिहला,786,...,शून्य,महेन्द्रगढ़,123029,082-भगड़ाना,"राजकीय हाई स्कूल,भगड़ाना (बांया कक्ष)",455,437,0,892,original
19196,CMB0690082.PDF,888,BTX1467190,मदन लाल,लालमन,िपता,225ख,68,पुरुष,787,...,शून्य,महेन्द्रगढ़,123029,082-भगड़ाना,"राजकीय हाई स्कूल,भगड़ाना (बांया कक्ष)",455,437,0,892,original
19197,CMB0690082.PDF,889,BTX1661560,शिमर्ला देवी,राजेश कुमार,पित,225ख,38,मिहला,788,...,शून्य,महेन्द्रगढ़,123029,082-भगड़ाना,"राजकीय हाई स्कूल,भगड़ाना (बांया कक्ष)",455,437,0,892,original
19198,CMB0690082.PDF,890,SHQ0875336,मनीष,बीर�दर् िसह,िपता,236,20,पुरुष,789,...,शून्य,महेन्द्रगढ़,123029,082-भगड़ाना,"राजकीय हाई स्कूल,भगड़ाना (बांया कक्ष)",455,437,0,892,original


In [34]:
df3[df3.elector_name_x==df3.elector_name_y]

,filename_x,number_x,id,elector_name_x,father_or_husband_name_x,relationship_x,house_no_x,age_x,sex_x,Unnamed: 0,...,revenue_division,district,pin_code,polling_station_name,polling_station_address,net_electors_male,net_electors_female,net_electors_third_gender,net_electors_total,original_or_amendment
69,CMB0880189.PDF,83,BFS2344570,महावीर,लाल राम,िपता,बी/90,39,पुरुष,79,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
75,CMB0880189.PDF,92,NAH0354324,औम प्रकाश,अमर िसह,िपता,बी156,38,पुरुष,88,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
863,CMB0880189.PDF,1112,NAH0895961,मंटू रजक,देव नारायण रजक,िपता,334,51,पुरुष,1031,...,शून्य,फरीदाबाद,121004,"189-आदर्श नगर, ऊँचा गांव","आर्दश विघा मन्दिर आदर्श नगर मलेरना रोड, बलल्‍...",753,955,0,1308,original
1398,CMB0550068.PDF,797,LYL1008085,मनवीर,रघु बीर,िपता,101/2,35,पुरुष,723,...,शून्य,भिवानी,127308,068-पिचोपा खुर्द,"राजकीय प्राईमरी स्कूल,पिचोपा खुर्द (पूर्वी भाग)",635,945,0,1180,original
1512,CMB0550068.PDF,980,LYL1040807,रामबीर,औमपर्काश,िपता,130,45,पुरुष,892,...,शून्य,भिवानी,127308,068-पिचोपा खुर्द,"राजकीय प्राईमरी स्कूल,पिचोपा खुर्द (पूर्वी भाग)",635,945,0,1180,original
2225,CMB0520102.PDF,587,CFH2051183,माया प्रसाद,िमठु पर्साद,िपता,127,44,पुरुष,525,...,26,हिसार,125001,102-कस्बा हिसार,"औधोगिक प्रशिक्षण संस्थान माडल टाऊन, हिसार (मध...",680,623,0,1303,original
3339,CMB0440155.PDF,797,JHY1159623,अमरजीत कौर,सुखदेविसह,पित,104,35,मिहला,728,...,शून्य,सिरसा,125055,155-अलीपुर-टीटू खेड़ा,"राजकीय प्राईमरी स्कूल, अलीपुर-टीटूखेड़ा",662,587,0,1249,original
3809,CMB0250129.PDF,317,NJK0525022,महावीर,मुन्शी राम,िपता,721,30,पुरुष,287,...,"24, 28",पानीपत,132103,129-पानीपत,"नवयुग पब्लिक स्कूल, मुखीजा कॉलोनी (मध्य दांया...",478,410,0,888,original
3859,CMB0250129.PDF,386,NJK1028794,गीता,मनोज,पित,765,25,मिहला,351,...,"24, 28",पानीपत,132103,129-पानीपत,"नवयुग पब्लिक स्कूल, मुखीजा कॉलोनी (मध्य दांया...",478,410,0,888,original
4346,CMB0560182.PDF,287,RFK0588103,संदीप,रमेश,िपता,85,26,पुरुष,261,...,शून्य,भिवानी,127306,182-बिगोवा,"राजकीय हाई स्कूल, बिगोवा (दक्षिणी भाग)",663,953,0,1216,original


In [35]:
df3.to_csv('/tmp/haryana.csv', index=False)